In [7]:
import scipy
import numpy as np
import scipy.spatial as spatial
import matplotlib.pyplot as plt
import random 
import pandas as pd

from mpl_toolkits.basemap import Basemap
from sklearn.datasets import fetch_species_distributions
from sklearn.neighbors import KernelDensity
import copy
from pyproj import Proj, transform

def convert_utm_to_latlon(df, zone_number, zone_letter):
    utm_proj = Proj(proj='utm', zone=zone_number, ellps='WGS84', south=(zone_letter < 'N'))
    lonlat_proj = Proj(proj='latlong', datum='WGS84')
    lon, lat = transform(utm_proj, lonlat_proj, df['Østkoordinat'].values, df['Nordkoordinat'].values)
    
    return pd.DataFrame({'Longitude': lon, 'Latitude': lat})


dat = pd.read_csv('C:/Users/vjosv/master/dataset/top185_in_oslo_area.csv')
dat = dat[['Id','Østkoordinat','Nordkoordinat','Vitenskapelig navn']]




dat['points']= [[i,j] for i, j in zip(dat.Østkoordinat, dat.Nordkoordinat)]

df_latlon = convert_utm_to_latlon(dat, 33, 'N') 


dat['lat']=df_latlon['Latitude']
dat['long']=df_latlon['Longitude']



#oslo area:
#lowerleft, upperleft, upper right, lower right,


lat_long_oslo = [(58.998141, 9.574585), (60.351413, 9.574585), (60.351413, 12.540894),(58.998141,12.540894)]

# dat['Østkoordinat']>lat_long_oslo[0][0]


dat = dat.loc[(dat['lat']>lat_long_oslo[0][0])  & (dat['lat']<lat_long_oslo[1][0]) & (dat['long']>lat_long_oslo[0][1 ]) & (dat['long']<lat_long_oslo[2][1])]





points = dat[['Østkoordinat','Nordkoordinat']].to_numpy()




point_tree = spatial.cKDTree(points)


science_names = dat['Vitenskapelig navn'].unique()
indexes = [i for i in range(len(dat['Vitenskapelig navn'].unique()))]


random.seed(10)

random.shuffle(indexes)


names_mapping = {science_name : index for (science_name,index) in zip(science_names,indexes)}
index_mapping = {index : science_name for (science_name,index) in zip(science_names,indexes)}

def distance_between_points(point, list_of_points):
    return [np.sqrt(np.power(point[0]-lop[0],2)+np.power(point[1]-lop[1],2)) for lop in list_of_points]



def get_points_within(df_row, distance=1000):
    id = int(df_row['Id'].iloc[0])
    return_list = point_tree.query_ball_point([[int(df_row['Østkoordinat'].iloc[0]),int(df_row['Nordkoordinat'].iloc[0])]], distance)[0]
    return_dat = dat.iloc[return_list]
    return_list = list(return_dat['Id'])
    return_list.remove(id)
    return return_list
    
def sample_plant_position(plant,df):
    if type(plant) == str:
        return dat[dat['Vitenskapelig navn'] == plant].sample(1)
    elif type(plant) == int:
        return dat[dat['Vitenskapelig navn'] == index_mapping[plant]].sample(1)

def kernel_density_estimate_value(point_row,dat,bandwidth = 0.5):
    # print(point_row)
    # print(dat)
    if point_row.index[0] in list(dat.index):
        np_dat_lat_long = dat.drop(point_row.index[0])[['lat','long']].to_numpy()
    else:
        np_dat_lat_long = dat[['lat','long']].to_numpy()
    kde = KernelDensity(bandwidth=bandwidth)
    # print(np_dat_lat_long)
    if len(np_dat_lat_long)==0:
        return 0
    kde.fit(np_dat_lat_long)
    np_point = np.array([[point_row['lat'].iloc[0],point_row['long'].iloc[0]]])
    return np.exp(kde.score_samples(np_point))[0]

def get_points_within_square(point, dat,side_length = 3000):
    return_dat = dat[dat['Østkoordinat']>point['Østkoordinat'].iloc[0]-side_length]
    return_dat = return_dat[return_dat['Østkoordinat']<point['Østkoordinat'].iloc[0]+ side_length]
    return_dat = return_dat[return_dat['Nordkoordinat']>point['Nordkoordinat'].iloc[0]- side_length]
    return_dat = return_dat[return_dat['Nordkoordinat']<point['Nordkoordinat'].iloc[0]+ side_length]
    return return_dat

# within_square =get_points_within_square(v,dat)

# within_square







def get_points_within(df_row, distance=1000):
    id = int(df_row['Id'].iloc[0])
    return_list = point_tree.query_ball_point([[int(df_row['Østkoordinat'].iloc[0]),int(df_row['Nordkoordinat'].iloc[0])]], distance)[0]
    return_dat = dat.iloc[return_list]
    return_list = list(return_dat['Id'])
    return_list.remove(id)
    return return_list
    
def sample_plant_position(plant,df):
    if type(plant) == str:
        return dat[dat['Vitenskapelig navn'] == plant].sample(1)
    elif type(plant) == int:
        return dat[dat['Vitenskapelig navn'] == index_mapping[plant]].sample(1)




def get_knn_classifier(samples_pos_list,dat,n=1000):
    sample_pos_indexes = [sp.index[0] for sp in samples_pos_list if sp.index[0] in list(dat.index)]
    
    dat_removed_samples = dat.drop(index=sample_pos_indexes)
    category = []
    for k in dat_removed_samples['Vitenskapelig navn']:
        category.append(names_mapping[k])
        
    category = np.array(category)
    points_np= np.array(list(dat_removed_samples['points']))
    oversample = imblearn.over_sampling.KMeansSMOTE()
    warnings.filterwarnings("ignore")
    X, y = oversample.fit_resample(points_np, category) 
    warnings.filterwarnings("default")
    
    neigh = KNeighborsClassifier(n_neighbors=n)
    neigh.fit(X, y)
    return neigh

C:\Users\vjosv\AppData\Local\Temp\ipykernel_10192\3690604550.py:22: DtypeWarning: Columns (14,15,24,25,26,37,38,39,40,46) have mixed types. Specify dtype option on import or set low_memory=False.
  dat = pd.read_csv('C:/Users/vjosv/master/dataset/top185_in_oslo_area.csv')
C:\Users\vjosv\AppData\Local\Temp\ipykernel_10192\3690604550.py:17: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(utm_proj, lonlat_proj, df['Østkoordinat'].values, df['Nordkoordinat'].values)
